In [4]:
#-*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.layers.core import Dropout
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,recall_score ,precision_score,f1_score
from sklearn.model_selection import StratifiedKFold,cross_validate
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
import os
import time
from tensorflow.python.keras.layers.core import Dropout

In [5]:
dataset_audience = pd.read_csv("D:/Project/2019 Lpoint/Code/Data/MERGE/dataset_audience.csv")
dataset_normal = pd.read_csv("D:/Project/2019 Lpoint/Code/Data/MERGE/normal_with_group.csv")
#dataset_purchase = pd.read_csv("D:/Project/2019 Lpoint/Code/Data/dataset_product.csv", encoding = 'unicode_escape')
dataset_purchase = pd.read_csv("D:/Project/2019 Lpoint/Code/recommend_algorithm/recommend/buying_list_clac3.csv")

In [6]:
dataset_purchase.head()

,clnt_id,Accessory Bags,Adult's Bed Covers and Skirts,Adult's Disposable Briefes,Adults' Bed Fillings,Adults' Bedding Sets,Adults' Duvet / Duvet Cover,Adults' Korean Style Mattress / Covers,Adults' Mattress Cover,Adults' Pads / Bed Spread,...,Yellow Corvina Fish,Yellow Corvina Fish Gift Sets,Yoga / Pilates Clothing,Yoga Mats / Exercise Mats,Yoghurts / Chunggukjang Makers,Yogurt Drinks,Young Pumpkins,Young Radishes,Zipper Poly Bags / Plastic Bags,kelp
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,40,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# dataset_purchase의 행수가 더 적으므로, 이를 기준으로 dataset_normal에서 추출
clnt_normal_list = dataset_purchase.clnt_id
dataset_normal = dataset_normal[dataset_normal.clnt_id.isin(clnt_normal_list)]

In [8]:
X = np.array(dataset_normal.drop(["clnt_id", "group"], axis=1))
y = np.array(dataset_purchase.drop("clnt_id", axis=1))

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.1, random_state=42)

In [9]:
def build_DNN():
    # checkpoint 저장 위치 설정
    MODEL_SAVE_FOLDER_PATH = './model/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)
    run_id = time.strftime("%y%m%d%H%M%S_")
    model_path = MODEL_SAVE_FOLDER_PATH + run_id + '{epoch:02d}-{val_loss:.4f}.h5'
    
    # checkpoint 설정
    checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                                verbose=1, save_best_only=True)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    
    # model
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(X_train.shape[1], activation='relu'))
    model.add(keras.layers.Dense(100, activation="relu"))
    model.add(keras.layers.Dense(100, activation="relu"))
    model.add(Dropout(0.2))
    model.add(keras.layers.Dense(80, activation="relu"))
    model.add(Dropout(0.2))
    model.add(keras.layers.Dense(50, activation="relu"))
    model.add(keras.layers.Dense(y_train.shape[1], activation="sigmoid"))
    
    # model compile
    model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.SGD(lr=0.005, momentum=0.9), metrics=["accuracy"])
    
    # history
    history = model.fit(X_train, y_train, epochs=200, validation_data = (X_valid, y_valid),
                   callbacks=[checkpoint_cb, early_stopping_cb])
    
    return model

In [10]:
model = build_DNN()

Train on 5340 samples, validate on 594 samples
Epoch 1/200
5152/5340 [===========================>..] - ETA: 0s - loss: 0.6925 - accuracy: 0.5233
Epoch 00001: val_loss improved from inf to 0.69085, saving model to ./model/200105131704_01-0.6909.h5
5340/5340 [==============================] - 3s 546us/sample - loss: 0.6925 - accuracy: 0.5246 - val_loss: 0.6909 - val_accuracy: 0.5747
Epoch 2/200
5312/5340 [============================>.] - ETA: 0s - loss: 0.6893 - accuracy: 0.5989
Epoch 00002: val_loss improved from 0.69085 to 0.68760, saving model to ./model/200105131704_02-0.6876.h5
5340/5340 [==============================] - 1s 225us/sample - loss: 0.6893 - accuracy: 0.5990 - val_loss: 0.6876 - val_accuracy: 0.6532
Epoch 3/200
5056/5340 [===========================>..] - ETA: 0s - loss: 0.6857 - accuracy: 0.6571
Epoch 00003: val_loss improved from 0.68760 to 0.68341, saving model to ./model/200105131704_03-0.6834.h5
5340/5340 [==============================] - 1s 209us/sample - loss:

Epoch 25/200
5312/5340 [============================>.] - ETA: 0s - loss: 0.0455 - accuracy: 0.9913
Epoch 00025: val_loss improved from 0.03841 to 0.03832, saving model to ./model/200105131704_25-0.0383.h5
5340/5340 [==============================] - 1s 159us/sample - loss: 0.0456 - accuracy: 0.9913 - val_loss: 0.0383 - val_accuracy: 0.9927
Epoch 26/200
4960/5340 [==========================>...] - ETA: 0s - loss: 0.0456 - accuracy: 0.9912
Epoch 00026: val_loss improved from 0.03832 to 0.03825, saving model to ./model/200105131704_26-0.0383.h5
5340/5340 [==============================] - 1s 164us/sample - loss: 0.0454 - accuracy: 0.9913 - val_loss: 0.0383 - val_accuracy: 0.9927
Epoch 27/200
5120/5340 [===========================>..] - ETA: 0s - loss: 0.0454 - accuracy: 0.9912
Epoch 00027: val_loss improved from 0.03825 to 0.03819, saving model to ./model/200105131704_27-0.0382.h5
5340/5340 [==============================] - 1s 158us/sample - loss: 0.0452 - accuracy: 0.9913 - val_loss: 0

Epoch 50/200
5152/5340 [===========================>..] - ETA: 0s - loss: 0.0440 - accuracy: 0.9913
Epoch 00050: val_loss improved from 0.03739 to 0.03738, saving model to ./model/200105131704_50-0.0374.h5
5340/5340 [==============================] - 1s 155us/sample - loss: 0.0439 - accuracy: 0.9913 - val_loss: 0.0374 - val_accuracy: 0.9927
Epoch 51/200
5184/5340 [============================>.] - ETA: 0s - loss: 0.0440 - accuracy: 0.9913
Epoch 00051: val_loss did not improve from 0.03738
5340/5340 [==============================] - 1s 144us/sample - loss: 0.0439 - accuracy: 0.9913 - val_loss: 0.0374 - val_accuracy: 0.9927
Epoch 52/200
5184/5340 [============================>.] - ETA: 0s - loss: 0.0437 - accuracy: 0.9913
Epoch 00052: val_loss did not improve from 0.03738
5340/5340 [==============================] - 1s 156us/sample - loss: 0.0437 - accuracy: 0.9913 - val_loss: 0.0374 - val_accuracy: 0.9927
Epoch 53/200
5120/5340 [===========================>..] - ETA: 0s - loss: 0.0436 

Epoch 77/200
5280/5340 [============================>.] - ETA: 0s - loss: 0.0430 - accuracy: 0.9913
Epoch 00077: val_loss did not improve from 0.03713
5340/5340 [==============================] - 1s 143us/sample - loss: 0.0431 - accuracy: 0.9913 - val_loss: 0.0371 - val_accuracy: 0.9927
Epoch 78/200
5056/5340 [===========================>..] - ETA: 0s - loss: 0.0431 - accuracy: 0.9913
Epoch 00078: val_loss did not improve from 0.03713
5340/5340 [==============================] - 1s 157us/sample - loss: 0.0433 - accuracy: 0.9913 - val_loss: 0.0372 - val_accuracy: 0.9927
Epoch 79/200
5184/5340 [============================>.] - ETA: 0s - loss: 0.0435 - accuracy: 0.9913
Epoch 00079: val_loss did not improve from 0.03713
5340/5340 [==============================] - 1s 170us/sample - loss: 0.0433 - accuracy: 0.9913 - val_loss: 0.0372 - val_accuracy: 0.9927


In [11]:
from keras.models import load_model
model.save("last_model.h5")

In [116]:
from keras.models import load_model
model = keras.models.load_model("last_model.h5")

In [12]:
model.predict(X_test)

array([[2.6822090e-07, 4.1127205e-06, 2.9802322e-08, ..., 8.3446503e-07,
        4.4107437e-05, 1.7881393e-07],
       [4.4703484e-07, 6.2584877e-06, 1.4901161e-07, ..., 1.1920929e-06,
        5.3584576e-05, 2.0861626e-07],
       [1.4901161e-05, 9.8258257e-05, 4.3511391e-06, ..., 3.5434961e-05,
        6.2894821e-04, 9.1791153e-06],
       ...,
       [1.2516975e-06, 1.3917685e-05, 2.0861626e-07, ..., 3.0994415e-06,
        1.0916591e-04, 6.2584877e-07],
       [7.6979399e-05, 3.8510561e-04, 2.7686357e-05, ..., 1.7145276e-04,
        1.9242465e-03, 5.4270029e-05],
       [2.3952127e-04, 1.0144413e-03, 9.6410513e-05, ..., 4.6600285e-04,
        3.8728369e-03, 1.7118883e-04]], dtype=float32)

In [13]:
model.evaluate(X_test, y_test)

1484/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.04105422165837249, 0.9916953]

In [17]:
X_test[0]

array([-0.12842312,  1.35339892,  0.80009933, -0.67449076, -0.67449076,
        1.3014875 ,  1.13090869, -1.        ,  0.32258065,  0.        ,
        0.256     ,  0.103     ,  0.128     ,  0.154     ,  0.051     ,
        0.205     ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])